<a href="https://colab.research.google.com/github/nata961/sistemas-de-informacion/blob/master/IAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importa las bibliotecas necesarias.

In [ ]:
import pandas as pd
from google.colab import drive


Montar Google Drive en Google Colab

In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


Definir la ruta de la carpeta y el nombre del archivo CSV

In [ ]:
import pandas as pd

# Definir la ruta del archivo CSV
file_path = '/content/drive/My Drive/Depuracion/357_depuracion.csv'

# Leer el archivo CSV en un DataFrame de Pandas
data = pd.read_csv(file_path, delimiter=';', encoding='latin1')

Leer la base de datos

In [ ]:
# Lee el archivo CSV en un DataFrame de Pandas
data = pd.read_csv(file_path, delimiter=';', encoding='latin1')

# Llenar los valores faltantes con cero en las columnas que deben ser enteras
columnas_entero = ['muestra', 'agente', 'resultado', 'lab_ajuste', 'area_', 'estrato_', 'gp_discapa',
                   'gp_desplaz', 'gp_migrant', 'gp_carcela', 'gp_gestan', 'gp_indigen', 'gp_pobicbf',
                   'gp_mad_com', 'gp_desmovi', 'gp_psiquia', 'gp_vic_vio', 'cod_dpto_r', 'cod_pais_r',
                   'cod_mun_r', 'caso_extra', 'crit_nav', 'crit_itsac', 'crit_istua', 'iad_polim', 'dias_prein']

data[columnas_entero] = data[columnas_entero].fillna(0)

# Convertir las columnas a tipo de dato entero
data[columnas_entero] = data[columnas_entero].astype(int)

# Verifica que el archivo se haya leído correctamente mostrando las primeras filas del DataFrame
print(data.head())

        control     fec_exa  fec_rec  muestra prueba  agente  resultado  \
0  LABORATORIOS  30/12/2023      NaN        2     G0     107          1   
1  LABORATORIOS  15/12/2023      NaN       32     C7    2260          1   
2  LABORATORIOS   8/12/2023      NaN        1     92    2287          1   
3  LABORATORIOS  12/01/2024      NaN        1     92     183          1   
4  LABORATORIOS   8/12/2023      NaN        1     92    2285          1   

      fec_exp valor  lab_ajuste  ...  \
0   1/01/2024   NaN           0  ...   
1  15/12/2023   NaN           0  ...   
2  12/12/2023   NaN           0  ...   
3  12/01/2024   NaN           0  ...   
4  10/12/2023   NaN           0  ...   

                                           nom_upgd npais_proce  ndep_proce  \
0        FUNDACION HOSPITALARIA SAN VICENTE DE PAUL    COLOMBIA   ANTIOQUIA   
1  CENTRO CARDIOVASCULAR COLOMBIANO CLINICA SANTA M    COLOMBIA   ANTIOQUIA   
2      PROMOTORA CLINICA ZONA FRANCA DE URABA   SAS    COLOMBIA   ANTIO

**Eliminar caracteres especiales **

In [ ]:
!pip install unidecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.3 MB/s eta 0:00:00


In [ ]:
from unidecode import unidecode
# Cambiar las letras 'Ñ' por 'N' y eliminar tildes en todas las columnas de texto
data = data.applymap(lambda x: unidecode(str(x)).replace('Ñ', 'N') if isinstance(x, str) else x)

Organizar encabezados de columnas

In [ ]:
print(data.columns)
data.columns = data.columns.str.strip()  # Eliminar espacios en blanco alrededor de los nombres de las columnas


Index(['control', 'fec_exa', 'fec_rec', 'muestra', 'prueba', 'agente',
       'resultado', 'fec_exp', 'valor', 'lab_ajuste',
       ...
       'nom_upgd', 'npais_proce', 'ndep_proce', 'nmun_proce', 'npais_resi',
       'ndep_resi', 'nmun_resi', 'ndep_notif', 'nmun_notif', 'FechaHora'],
      dtype='object', length=152)


Paso 1: Eliminar los casos con departamento diferente de Antioquia

In [ ]:
import pandas as pd

# Filtrar por la variable 'ndep_notif' para mantener solo los casos notificados en Antioquia
data_antioquia = data[data['ndep_notif'] == 'ANTIOQUIA']

# Ajustar la precisión de la visualización de los datos numéricos
pd.set_option('display.float_format', '{:.2f}'.format)

# Verificar la cantidad de registros después de filtrar por Antioquia
print("Cantidad de registros después de filtrar por Antioquia:", len(data_antioquia))

# Guardar la base de datos filtrada en un nuevo archivo CSV
file_path_antioquia = '/content/drive/My Drive/Depuracion/data_antioquia.csv'
data_antioquia.to_csv(file_path_antioquia, index=False)

print("Base de datos filtrada por Antioquia guardada en:", file_path_antioquia)


Cantidad de registros después de filtrar por Antioquia: 409
Base de datos filtrada por Antioquia guardada en: /content/drive/My Drive/Depuracion/data_antioquia.csv


Paso 2: Separar los descartados en una base de datos diferente

In [ ]:
# Filtrar los casos con ajuste 'D' o '6' en los datos de Antioquia
descartados_antioquia = data_antioquia[(data_antioquia['ajuste_'] == 'D') | (data_antioquia['ajuste_'] == '6')]

# Verificar la cantidad de registros a descartar en Antioquia
print("Cantidad de registros a descartar en Antioquia:", len(descartados_antioquia))

# Eliminar los casos descartados de los datos de Antioquia
data_antioquia = data_antioquia.drop(descartados_antioquia.index)

# Verificar la cantidad de registros en Antioquia después de retirar los descartados
print("Cantidad de registros en Antioquia después de retirar los descartados:", len(data_antioquia))

# Guardar los casos descartados de Antioquia en otra pestaña llamada Descartados
descartados_file_path = '/content/drive/My Drive/Depuracion/descartados_antioquia.csv'
descartados_antioquia.to_csv(descartados_file_path, index=False)

print("Casos descartados en Antioquia guardados en el archivo:", descartados_file_path)

Cantidad de registros a descartar en Antioquia: 22
Cantidad de registros en Antioquia después de retirar los descartados: 387
Casos descartados en Antioquia guardados en el archivo: /content/drive/My Drive/Depuracion/descartados_antioquia.csv


PASO 3: Eliminar los ajustes por laboratorio con ajuste 6 para el criterio NAV 2 y 3

In [ ]:
# Filtrar los casos con 'crit_nav' igual a 2 o 3 en los datos de Antioquia
crit_nav_2_3 = data_antioquia[(data_antioquia['crit_nav'] == 2) | (data_antioquia['crit_nav'] == 3)]

# Filtrar los casos de crit_nav_2_3 que también tienen lab_ajuste igual a 6
descartados_crit_nav_6 = crit_nav_2_3[crit_nav_2_3['lab_ajuste'] == 6]

# Guardar los casos descartados en otra base de datos
descartados_file_path = '/content/drive/My Drive/Depuracion/descartados_lab6_crit23_antioquia.csv'
descartados_crit_nav_6.to_csv(descartados_file_path, index=False)

print("Casos descartados de laboratorio ajuste 6 y criterios 2 y 3 guardados en:", descartados_file_path)

# Eliminar los casos filtrados de data_antioquia
data_antioquia = data_antioquia.drop(descartados_crit_nav_6.index)

# Verificar la cantidad de registros en data_antioquia después de eliminar los casos
print("Cantidad de registros en data_antioquia después de eliminar los casos:", len(data_antioquia))

Casos descartados de laboratorio ajuste 6 y criterios 2 y 3 guardados en: /content/drive/My Drive/Depuracion/descartados_lab6_crit23_antioquia.csv
Cantidad de registros en data_antioquia después de eliminar los casos: 380


PASO 4: Eliminar los ajustes de laboratorio 6 y tipo iad 2 y 3

In [ ]:
# Filtrar los casos con 'tipo_iad' igual a 2 o 3 en los datos de Antioquia
tipo_iad_2_3 = data_antioquia[(data_antioquia['tipo_iad'] == 2) | (data_antioquia['tipo_iad'] == 3)]

# Filtrar los casos de tipo_iad_2_3 que también tienen lab_ajuste igual a 6
descartados_tipo_iad_6 = tipo_iad_2_3[tipo_iad_2_3['lab_ajuste'] == 6]

# Guardar los casos descartados en otra base de datos
descartados_file_path = '/content/drive/My Drive/Depuracion/descartados_lab6_tipoiad23_antioquia.csv'
descartados_tipo_iad_6.to_csv(descartados_file_path, index=False)

print("Casos descartados de laboratorio ajuste 6 y tipo IAD 2 y 3 guardados en:", descartados_file_path)

# Eliminar los casos filtrados de data_antioquia
data_antioquia = data_antioquia.drop(descartados_tipo_iad_6.index)

# Verificar la cantidad de registros en data_antioquia después de eliminar los casos
print("Cantidad de registros en data_antioquia después de eliminar los casos:", len(data_antioquia))

Casos descartados de laboratorio ajuste 6 y tipo IAD 2 y 3 guardados en: /content/drive/My Drive/Depuracion/descartados_lab6_tipoiad23_antioquia.csv
Cantidad de registros en data_antioquia después de eliminar los casos: 373


PASO 5= Eliminar ajuste de laboratorio 6 y tipo IAD 1 y Crit Nav 1, desde la columna A a la I

In [ ]:
# Filtrar los casos con lab_ajuste igual a 6, tipo_iad igual a 1 y crit_nav igual a 1
casos_a_borrar = data_antioquia[(data_antioquia['lab_ajuste'] == 6) &
                                 (data_antioquia['tipo_iad'] == 1) &
                                 (data_antioquia['crit_nav'] == 1)]

# Guardar la cantidad de casos encontrados
cantidad_casos_borrados = len(casos_a_borrar)
print("Cantidad de casos encontrados:", cantidad_casos_borrados)

# Eliminar las columnas desde la A hasta la I para los casos filtrados
columnas_a_borrar = data_antioquia.columns[:9]  # Seleccionar las columnas de la A a la I
data_antioquia.loc[casos_a_borrar.index, columnas_a_borrar] = None

# Verificar la cantidad de registros en data_antioquia después de eliminar los casos
print("Cantidad de registros en data_antioquia después de eliminar los casos:", len(data_antioquia))

Cantidad de casos encontrados: 4
Cantidad de registros en data_antioquia después de eliminar los casos: 373


PASO 6= Retirar fecha diagnostico que sean del año anterior

PASO 7: Para todo paciente que es remitido remitido =1  calcular la diferencia entre fec_diag y fecha de ingreso a la UCI fec_inguc si la diferencia es mayor o igual a 3 debe ser intrahospitalario, caso_extra = 2 . Clasificar los casos como extrahospitalario caso_extra = 1 aquellos cuya diferencia sea menor a 3 y copiarlos en otra pestaña y retirarlos de la base. Aquellos cuya diferencia es un valor negativo se debe solicitar revisión de la inconsistencia.


In [ ]:
import pandas as pd
# Convertir las columnas 'fec_diag' y 'fec_inguci' al formato de fecha
data_antioquia['fec_inguci'] = pd.to_datetime(data_antioquia['fec_inguci'], format='%d/%m/%Y', errors='coerce')
data_antioquia['fec_diag'] = pd.to_datetime(data_antioquia['fec_diag'], format='%d/%m/%Y', errors='coerce')

# Filtrar los casos donde 'remitido' es igual a 1
casos_remitidos = data_antioquia[data_antioquia['remitido'] == 1]

# Filtrar los casos donde 'caso_extra' es igual a 2
casos_caso_extra_2 = casos_remitidos[casos_remitidos['caso_extra'] == 2]

# Calcular la diferencia entre 'fec_diag' y 'fec_inguci'
diferencia_timedelta = casos_caso_extra_2['fec_diag'] - casos_caso_extra_2['fec_inguci']

# Actualizar la columna 'AJUSTE' para aquellos casos donde la resta es negativa o menor a 3 días
data_antioquia.loc[(data_antioquia['remitido'] == 1) & (data_antioquia['caso_extra'] == 2) & (diferencia_timedelta < pd.Timedelta(days=3)), 'AJUSTE'] = 'Intrahospitalario'

# Actualizar la columna 'AJUSTE' para aquellos casos donde la resta es negativa
data_antioquia.loc[(data_antioquia['remitido'] == 1) & (data_antioquia['caso_extra'] == 2) & (diferencia_timedelta < pd.Timedelta(days=0)), 'AJUSTE'] = 'Inconsistencia entre fecdiag y fecinguci'

# Guardar la base de datos con las nuevas columnas
data_antioquia.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

# Verificar la cantidad de registros en data_antioquia después de eliminar los casos
print("Cantidad de registros en data_antioquia después de eliminar los casos:", len(data_antioquia))

Cantidad de registros en data_antioquia después de eliminar los casos: 373


In [ ]:
import pandas as pd

# Filtrar los casos donde 'remitido' es igual a 1 y 'caso_extra' es igual a 1
casos_remitidos_caso_extra_1 = data_antioquia[(data_antioquia['remitido'] == 1) & (data_antioquia['caso_extra'] == 1)]

# Calcular la diferencia entre 'fec_diag' y 'fec_inguci'
diferencia_timedelta = casos_remitidos_caso_extra_1['fec_diag'] - casos_remitidos_caso_extra_1['fec_inguci']

# Actualizar la columna 'AJUSTE' para aquellos casos donde la resta es mayor o igual a 3 días
data_antioquia.loc[(data_antioquia['remitido'] == 1) & (data_antioquia['caso_extra'] == 1) & (diferencia_timedelta >= pd.Timedelta(days=3)), 'AJUSTE'] = 'Extrahospitalario'

# Actualizar la columna 'AJUSTE' para aquellos casos donde la resta es negativa
data_antioquia.loc[(data_antioquia['remitido'] == 1) & (data_antioquia['caso_extra'] == 1) & (diferencia_timedelta < pd.Timedelta(days=0)), 'AJUSTE'] = 'Inconsistencia entre fecdiag y fecinguci'

# Guardar la base de datos con las nuevas columnas
data_antioquia.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

# Verificar la cantidad de registros en data_antioquia después de eliminar los casos
print("Cantidad de registros en data_antioquia después de eliminar los casos:", len(data_antioquia))


Cantidad de registros en data_antioquia después de eliminar los casos: 373


PASO 8: INCONSISTENCIA EN FECHAS

In [ ]:
import pandas as pd

# Agregar la columna 'AJUSTEFECHAS' con valores predeterminados vacíos
data_antioquia['AJUSTEFECHAS'] = ''

# Filtrar los casos donde alguna de las fechas sea mayor que la fec_diag
fechas_mayores_fec_diag = data_antioquia[(data_antioquia['fec_con_'] > data_antioquia['fec_diag']) |
                                          (data_antioquia['ini_sin_'] > data_antioquia['fec_diag']) |
                                          (data_antioquia['fec_hos_'] > data_antioquia['fec_diag'])]

# Si hay casos donde alguna de las fechas es mayor que fec_diag, ajustar la columna 'AJUSTE'
if not fechas_mayores_fec_diag.empty:
    # Ajustar la columna 'AJUSTEFECHAS' para indicar que se debe ajustar la concordancia de fechas
    data_antioquia.loc[fechas_mayores_fec_diag.index, 'AJUSTEFECHAS'] = 'Ajustar concordancia de fechas fec_con_; ini_sin_; fec_hos'

# Guardar la base de datos con las nuevas columnas
data_antioquia.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

# Verificar la cantidad de registros en data_antioquia después de eliminar los casos
print("Cantidad de registros en data_antioquia después de eliminar los casos:", len(data_antioquia))

Cantidad de registros en data_antioquia después de eliminar los casos: 373


PASO 9: CONCORDANCIA FECHA DEFUNCION

In [ ]:
import pandas as pd
# Convertir las columnas 'fec_diag' y 'fec_inguci' al formato de fecha
data_antioquia['fec_def_'] = pd.to_datetime(data_antioquia['fec_def_'], format='%d/%m/%Y', errors='coerce')
# Agregar la columna 'AJUSTEFECHAS' con valores predeterminados vacíos
data_antioquia['AJUSTEDEFUNCION'] = ''

fechas_mayores_fec_diag = data_antioquia[(data_antioquia['fec_def_'] < data_antioquia['fec_diag'])]

# Si hay casos donde alguna de las fechas es mayor que fec_diag, ajustar la columna 'AJUSTE'
if not fechas_mayores_fec_diag.empty:
    # Ajustar la columna 'AJUSTEFECHAS' para indicar que se debe ajustar la concordancia de fechas
    data_antioquia.loc[fechas_mayores_fec_diag.index, 'AJUSTEDEFUNCION'] = 'Ajustar concordancia de fec_def'

# Guardar la base de datos con las nuevas columnas
data_antioquia.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

# Verificar la cantidad de registros en data_antioquia después de eliminar los casos
print("Cantidad de registros en data_antioquia después de eliminar los casos:", len(data_antioquia))

Cantidad de registros en data_antioquia después de eliminar los casos: 373


PASO 10: Concordancia fecha inicio sintomas y fecha hospitalizacion

In [ ]:
import pandas as pd

# Agregar la columna 'AJUSTEFECHAS' con valores predeterminados vacíos
data_antioquia['AJUSTESINTOMAS'] = ''

fechas_mayores_fec_diag = data_antioquia[(data_antioquia['ini_sin_'] < data_antioquia['fec_hos_'])]

# Si hay casos donde alguna de las fechas es mayor que fec_diag, ajustar la columna 'AJUSTE'
if not fechas_mayores_fec_diag.empty:
    # Ajustar la columna 'AJUSTEFECHAS' para indicar que se debe ajustar la concordancia de fechas
    data_antioquia.loc[fechas_mayores_fec_diag.index, 'AJUSTESINTOMAS'] = 'Ajustar concordancia de inicio de sintomas'

# Guardar la base de datos con las nuevas columnas
data_antioquia.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

# Verificar la cantidad de registros en data_antioquia después de eliminar los casos
print("Cantidad de registros en data_antioquia después de eliminar los casos:", len(data_antioquia))

Cantidad de registros en data_antioquia después de eliminar los casos: 373


PASO 11: Fecha diagnostico igual a la fecha examen

In [ ]:
import pandas as pd

# Agregar la columna 'AJUSTEFECHAS' con valores predeterminados vacíos
data_antioquia['AJUSTEFECEXAMEN'] = ''

# Filtrar filas donde control sea "LABORATORIO" y fec_exa no sea nulo
data_antioquia_lab = data_antioquia[(data_antioquia['control'] == 'LABORATORIOS') & data_antioquia['fec_exa'].notnull()]

# Filtrar filas donde fec_diag no sea nulo
data_antioquia_lab = data_antioquia_lab[data_antioquia_lab['fec_diag'].notnull()]

# Identificar las filas donde las fechas de fec_exa y fec_diag son diferentes
fechas_diferentes = data_antioquia_lab[data_antioquia_lab['fec_exa'] != data_antioquia_lab['fec_diag']]

# Añadir la columna AJUSTEFECEXAMEN y asignar el mensaje "AJUSTAR FECHA EXAMEN" a las filas donde las fechas son diferentes
data_antioquia.loc[fechas_diferentes.index, 'AJUSTEFECEXAMEN'] = 'AJUSTAR FECHA EXAMEN'

# Guardar la base de datos con la columna de ajustes
data_antioquia.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

# Verificar la cantidad de registros en fechas_diferentes
print("Cantidad de registros con fechas diferentes:", len(fechas_diferentes))

Cantidad de registros con fechas diferentes: 261


PASO 12: CASOS QUE NO TIENE EXAMENES Y CRIT_NAV 1



In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Agregar la columna 'AJUSTEFECHAS' con valores predeterminados vacíos
data_antioquia_con_ajuste['AJUSTECRITNAV1LABORATORIO'] = ''

# Filtrar los casos donde 'crit_nav' es diferente de 1 y control no es nulo
filtro = (data_antioquia_con_ajuste['control'].isnull()) & \
         (data_antioquia_con_ajuste['crit_nav'] != 1)

# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1LABORATORIO'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTECRITNAV1LABORATORIO'] = 'Solicitar revisión de laboratorio a la entidad territorial'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)


PASO 13: Todos los datos de la variable paciente hospitalizado debe ser 1

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Agregar la columna 'AJUSTEFECHAS' con valores predeterminados vacíos
data_antioquia_con_ajuste['AJUSTEHOSPITALIZADO'] = ''

# Filtrar los casos donde 'crit_nav' es diferente de 1 y control no es nulo
filtro = (data_antioquia_con_ajuste['pac_hos_'] != 1)

# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1LABORATORIO'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTEHOSPITALIZADO'] = 'Todos los pacientes deben ser pac_hos 1 '

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 14:

Condición 1

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_uci' es diferente de 3, 'uni_med_' es 2 y 'edad_' es menor que 4
filtro = (data_antioquia_con_ajuste['tipo_uci'] != 3) & (data_antioquia_con_ajuste['uni_med_'] == 2) & (data_antioquia_con_ajuste['edad_'] < 4)

# Crear una nueva columna 'AJUSTEUCI3' con valor predeterminado de ''
data_antioquia_con_ajuste['AJUSTEUCI3'] = ''

# Actualizar la columna 'AJUSTEUCI3' para aquellos casos que cumplan con el filtro
data_antioquia_con_ajuste.loc[filtro, 'AJUSTEUCI3'] = 'Ajustar a tipo de uci 3'

# Guardar la base de datos actualizada con la nueva columna
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)


Condición 2

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_uci' es diferente de 3 y 'uni_med_' es 3
filtro = (data_antioquia_con_ajuste['tipo_uci'] != 3) & (data_antioquia_con_ajuste['uni_med_'] == 3)

# Crear una nueva columna 'AJUSTEUCI3UNI3' con valor predeterminado de ''
data_antioquia_con_ajuste['AJUSTEUCI3UNI3'] = ''

# Actualizar la columna 'AJUSTEUCI3UNI3' para aquellos casos que cumplan con el filtro
data_antioquia_con_ajuste.loc[filtro, 'AJUSTEUCI3UNI3'] = 'Ajustar a tipo de uci 3'

# Guardar la base de datos actualizada con la nueva columna
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

Condición 3

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_uci' es diferente de 2, 'uni_med_' es 2 y 'edad_' es mayor que 3
filtro = (data_antioquia_con_ajuste['tipo_uci'] != 2) & (data_antioquia_con_ajuste['uni_med_'] == 2) & (data_antioquia_con_ajuste['edad_'] > 3)

# Crear una nueva columna 'AJUSTEUCI2' con valor predeterminado de ''
data_antioquia_con_ajuste['AJUSTEUCI2'] = ''

# Actualizar la columna 'AJUSTEUCI2' para aquellos casos que cumplan con el filtro
data_antioquia_con_ajuste.loc[filtro, 'AJUSTEUCI2'] = 'Ajustar a tipo de uci 2'

# Guardar la base de datos actualizada con la nueva columna
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

Condición 4

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_uci' es diferente de 2, 'uni_med_' es 1 y 'edad_' es menor que 14
filtro = (data_antioquia_con_ajuste['tipo_uci'] != 2) & (data_antioquia_con_ajuste['uni_med_'] == 1) & (data_antioquia_con_ajuste['edad_'] < 14)

# Crear una nueva columna 'AJUSTEUCI2mayor14' con valor predeterminado de ''
data_antioquia_con_ajuste['AJUSTEUCI2mayor14'] = ''

# Actualizar la columna 'AJUSTEUCI2mayor14' para aquellos casos que cumplan con el filtro
data_antioquia_con_ajuste.loc[filtro, 'AJUSTEUCI2mayor14'] = 'Ajustar a tipo de uci 2'

# Guardar la base de datos actualizada con la nueva columna
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

CONDICION 5:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_uci' es diferente de 1, 'uni_med_' es 1 y 'edad_' es mayor o igual a 18
filtro = (data_antioquia_con_ajuste['tipo_uci'] != 1) & (data_antioquia_con_ajuste['uni_med_'] == 1) & (data_antioquia_con_ajuste['edad_'] >= 18)

# Crear una nueva columna 'AJUSTEUCI1' con valor predeterminado de ''
data_antioquia_con_ajuste['AJUSTEUCI1'] = ''

# Actualizar la columna 'AJUSTEUCI1' para aquellos casos que cumplan con el filtro
data_antioquia_con_ajuste.loc[filtro, 'AJUSTEUCI1'] = 'Ajustar a tipo de uci 1'

# Guardar la base de datos actualizada con la nueva columna
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 15:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 1
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 1) & (data_antioquia_con_ajuste['crit_nav'] == 1)

# Filtrar los casos que tengan 'muestra' igual a 10, 31 o 32
filtro_muestra = data_antioquia_con_ajuste['muestra'].isin([10, 31, 32])

# Aplicar el filtro
casos_filtrados = data_antioquia_con_ajuste[filtro & filtro_muestra]

# Guardar los casos filtrados en otra base de datos
casos_filtrados.to_csv('/content/drive/My Drive/Depuracion/casos_filtrados.csv', index=False)

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 16:

PASO 17: Filtrar tipo_iad = 1 + crit_nav = 2 y 3 >> muestra = null >> agente = null  se reclasifica a crit_nav = 1.

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Agregar la columna 'AJUSTE' con valores predeterminados vacíos
data_antioquia_con_ajuste['AJUSTECRITNAV1'] = ''

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3
filtro_tipo_iad = data_antioquia_con_ajuste['tipo_iad'] == 1
filtro_crit_nav = data_antioquia_con_ajuste['crit_nav'].isin([2, 3])

# Filtrar los casos donde 'muestra' y 'agente' son nulos
filtro_muestra_agente = data_antioquia_con_ajuste['muestra'].isnull() & data_antioquia_con_ajuste['agente'].isnull()

# Aplicar el filtro
casos_filtrados = data_antioquia_con_ajuste[filtro_crit_nav & filtro_muestra_agente]

# Crear la columna 'AJUSTECRITNAV1' y asignar el comentario correspondiente
casos_filtrados['AJUSTECRITNAV1'] = 'Ajustar a criterio NAV 1'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 18:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 1) & \
         (data_antioquia_con_ajuste['crit_nav'].isin([2, 3])) & \
         (data_antioquia_con_ajuste['muestra'].isin([10, 32]))

# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1.1'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTECRITNAV1.1'] = 'Ajustar a criterio NAV 1'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 19:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 1) & \
         (data_antioquia_con_ajuste['crit_nav'] == 2) & \
         (data_antioquia_con_ajuste['inmunosupr'] == 1)

# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1.1'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTECRITNAV3'] = 'NAV criterio 2 en paciente inmunosuprimido, ajustar a criterio NAV 3'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 20:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 2) & \
         (data_antioquia_con_ajuste['muestra'].isin([10, 32, 11]))

# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1.1'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTEMUESTRA'] = 'Ajustar las muestras inapropiadas'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 21:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 2) & \
         (data_antioquia_con_ajuste['crit_istua'].isin([1, 2,])) & \
         (data_antioquia_con_ajuste['uni_med_'].isin([2, 3,]))

# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1.1'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTECRITISTUA2-4'] = 'Ajustar reclasificación a crit 3 o 4, según corresponda'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 22:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 2) & \
         (data_antioquia_con_ajuste['crit_istua'].isin([3, 4,])) & \
         (data_antioquia_con_ajuste['uni_med_'] == 1)

# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1.1'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTECRITISTUA1-2'] = 'Ajustar reclasificación a crit 1 o 2, según corresponda'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 23:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 3) & \
         (data_antioquia_con_ajuste['muestra'] != 1) & \
         (data_antioquia_con_ajuste['prueba'] != 92)

# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1.1'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTEMUESTRAPRUEBA'] = 'Ajustar muestra y prueba'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)


PASO 24:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Definir los agentes a excluir
agentes_excluidos = [170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 381, 477, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 1854, 2500, 3428, 3429, 3430, 3431, 3432, 3433, 3434, 3435, 3436, 3437, 3438, 3439, 3440, 3441, 3442, 3443, 3444, 3445, 3446, 3447, 3448, 3449, 3450, 3451, 3452, 3453, 3454, 3455, 3456, 3457, 3458, 3459, 3460, 3461, 3462, 3463, 3464, 3465, 3466, 3467, 3468, 203, 204, 205]

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 3) & \
         (data_antioquia_con_ajuste['crit_itsac'].isin([2, 3,])) & \
         (~data_antioquia_con_ajuste['agente'].isin(agentes_excluidos))

# Asignar el comentario correspondiente a las filas seleccionadas
data_antioquia_con_ajuste.loc[filtro, 'AJUSTECRITITSAC1'] = 'Ajustar a crit itsac 1'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 25:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 3) & \
         (data_antioquia_con_ajuste['crit_itsac'] == 1) & \
         (data_antioquia_con_ajuste['uni_med_'] != 1) & \
         (data_antioquia_con_ajuste['agente'] == 183)


# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1.1'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTECRITITSAC3'] = 'Ajustar a crit itsac 3'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 26:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 3) & \
         (data_antioquia_con_ajuste['crit_itsac'] == 2) & \
         (data_antioquia_con_ajuste['uni_med_'] != 1)



# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1.1'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTECRITITSAC3.1'] = 'Ajustar a crit itsac 3'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

PASO 27:

In [ ]:
import pandas as pd

# Cargar la base de datos con ajustes
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Filtrar los casos donde 'tipo_iad' es igual a 1 y 'crit_nav' es igual a 2 o 3, y 'muestra' es 10 o 32
filtro = (data_antioquia_con_ajuste['tipo_iad'] == 3) & \
         (data_antioquia_con_ajuste['crit_itsac'] == 3) & \
         (data_antioquia_con_ajuste['uni_med_'] == 1)

# Asignar el comentario correspondiente en la columna 'AJUSTECRITNAV1.1'
data_antioquia_con_ajuste.loc[filtro, 'AJUSTECRITITSAC2'] = 'Ajustar a crit itsac 2'

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)


PASO 28:

In [ ]:
# Lista de las columnas que deseas mantener visibles
columnas_visibles = [
    'fec_exa', 'muestra', 'prueba', 'agente', 'fec_not',
    'pri_nom_', 'seg_nom_', 'pri_ape_', 'seg_ape_', 'num_ide_',
    'edad_', 'ini_sin_', 'tip_cas_', 'pac_hos_', 'fec_hos_',
    'con_fin_', 'fec_inguci', 'tipo_iad', 'crit_nav',
    'crit_itsac', 'crit_istua', 'fec_diag', 'iad_polim'
]

# Ocultar todas las columnas excepto las visibles
data_antioquia_final = data_antioquia_con_ajuste.loc[:, columnas_visibles]

# Crear una copia del DataFrame
data_antioquia_con_ajuste_copy = data_antioquia_con_ajuste.copy()

# Guardar la copia en Google Drive con el nombre aAJUTES
data_antioquia_con_ajuste_copy.to_csv('/content/drive/My Drive/Depuracion/AJUSTES.csv', index=False)

# Guardar la base de datos filtrada como data_antioquia_final
data_antioquia_final.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_final.csv', index=False)


PASO 29:

In [ ]:
import pandas as pd

# Leer la base de datos original, especificando que la columna 'num_ide' debe ser leída como cadena de caracteres
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Identificar duplicados basados en las columnas num_ide, tipo_iad, fec_diag y microorganismo
duplicados = data_antioquia_con_ajuste[data_antioquia_con_ajuste.duplicated(subset=['num_ide_', 'tipo_iad', 'fec_diag', 'agente'], keep=False)]

# Crear una base de datos para los duplicados
duplicados.to_csv('/content/drive/My Drive/Depuracion/duplicados.csv', index=False)

# Eliminar los duplicados de la base de datos original
data_antioquia_con_ajuste.drop_duplicates(subset=['num_ide_', 'tipo_iad', 'fec_diag', 'agente'], keep='first', inplace=True)

# Guardar la base de datos actualizada
data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

# Verificar la cantidad de registros después de filtrar por Antioquia
print("Cantidad de registros después de filtrar por Antioquia:", len(data_antioquia_con_ajuste))

Cantidad de registros después de filtrar por Antioquia: 372


Condición 2:

In [ ]:
import pandas as pd

# Leer la base de datos original
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Identificar duplicados basados en las columnas num_ide, tipo_iad y fec_diag
duplicados = data_antioquia_con_ajuste[data_antioquia_con_ajuste.duplicated(subset=['num_ide_', 'tipo_iad', 'fec_diag' , 'agente'], keep=False)]



# Asegúrate de que la columna 'fec_diag' esté en formato datetime
duplicados['fec_diag'] = pd.to_datetime(duplicados['fec_diag'], errors='coerce')

# Ahora aplica tu filtro
filtro_fec_diag = (duplicados['fec_diag'].diff().dt.days.abs() < 14) | (duplicados['fec_diag'].diff().isna())

# Aplicar el filtro
duplicados_filtrados = duplicados[filtro_fec_diag]

# Verificar si hay duplicados filtrados
if not duplicados_filtrados.empty:
    # Eliminar los duplicados de la base de datos original
    data_antioquia_con_ajuste.drop_duplicates(subset=['num_ide_', 'tipo_iad', 'fec_diag' , 'agente'], keep='first', inplace=True)

    # Guardar la base de datos actualizada
    data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

    # Crear una base de datos para los duplicados
    duplicados_filtrados.to_csv('/content/drive/My Drive/Depuracion/duplicados_agente.csv', index=False)

    print("Duplicados encontrados y eliminados. Se han guardado en un archivo separado.")
else:
    print("No se encontraron duplicados que cumplan con los criterios de filtro.")


# Verificar la cantidad de registros después de filtrar por Antioquia
print("Cantidad de registros después de filtrar por Antioquia:", len(data_antioquia_con_ajuste))


No se encontraron duplicados que cumplan con los criterios de filtro.
Cantidad de registros después de filtrar por Antioquia: 372


PASO 30:

In [ ]:
import pandas as pd

# Leer la base de datos original
data_antioquia_con_ajuste = pd.read_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv')

# Identificar duplicados basados en las columnas num_ide, tipo_iad y fec_diag
duplicados = data_antioquia_con_ajuste[data_antioquia_con_ajuste.duplicated(subset=['num_ide_', 'tipo_iad', 'fec_diag'], keep=False)]


# Asegúrate de que la columna 'fec_diag' esté en formato datetime
duplicados['fec_diag'] = pd.to_datetime(duplicados['fec_diag'], errors='coerce')

# Ahora aplica tu filtro
filtro_fec_diag = (duplicados['fec_diag'].diff().dt.days.abs() < 14) | (duplicados['fec_diag'].diff().isna())

# Aplicar el filtro
duplicados_filtrados = duplicados[filtro_fec_diag]

# Verificar si hay duplicados filtrados
if not duplicados_filtrados.empty:
    # Eliminar los duplicados de la base de datos original
    data_antioquia_con_ajuste.drop_duplicates(subset=['num_ide_', 'tipo_iad', 'fec_diag'], keep='first', inplace=True)

    # Guardar la base de datos actualizada
    data_antioquia_con_ajuste.to_csv('/content/drive/My Drive/Depuracion/data_antioquia_con_ajuste.csv', index=False)

    # Crear una base de datos para los duplicados
    duplicados_filtrados.to_csv('/content/drive/My Drive/Depuracion/duplicados_tipo_3.csv', index=False)

    print("Duplicados encontrados y eliminados. Se han guardado en un archivo separado.")
else:
    print("No se encontraron duplicados que cumplan con los criterios de filtro.")


# Verificar la cantidad de registros después de filtrar por Antioquia
print("Cantidad de registros después de filtrar por Antioquia:", len(data_antioquia_con_ajuste))


Duplicados encontrados y eliminados. Se han guardado en un archivo separado.
Cantidad de registros después de filtrar por Antioquia: 358


<ipython-input-41-1a27f097c2fc>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicados['fec_diag'] = pd.to_datetime(duplicados['fec_diag'], errors='coerce')
